In [1]:
import numpy as np
import pandas as pd


In [2]:
import genetic_alg as ga

In [3]:
from dataset import load_dataset, load_labels, convert_to_epochs, load_channels
from features import time_series_features, nonlinear_features, entropy_features, hjorth_features, freq_band_features
from classifiers import LR, KNN, SVM, NN
import variables as v

In [4]:
#importing ICA filtered two times

dataset_ica_2_ = load_dataset(data_type="ica2", test_type="Arithmetic")

channels = load_channels()

print(channels)

['Cz', 'Fz', 'Fp1', 'F7', 'F3', 'FC1', 'C3', 'FC5', 'FT9', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO9', 'O1', 'Pz', 'Oz', 'O2', 'PO10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'FT10', 'FC6', 'C4', 'FC2', 'F4', 'F8', 'Fp2']


In [5]:
channels_0  = ['F3', 'C3', 'P3', 'O1', 'O2', 'P4', 'C4', 'F4']
channels_1  = ['Fp1', 'C3', 'P7', 'O1', 'O2', 'P8', 'C4', 'Fp2']
channels_2  = ['Cz', 'Fz', 'P3', 'Pz', 'O1', 'Oz', 'O2', 'P4']
channels_3  = ['Cz', 'Fz', 'T7', 'P3', 'O1', 'O2', 'P4', 'T8']
channels_4  = ['F7', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'F8']
channels_5  = ['FC1', 'FC5', 'FT9', 'CP5', 'CP1', 'PO9', 'T8', 'C4']
channels_6  = ['F3', 'P7', 'PO10', 'CP2', 'CP6', 'FT10', 'FC6', 'FC2']
channels_7  = ['Fz', 'F7', 'FC1', 'T7', 'Pz', 'P8','FC6','Fp2']
channels_8  = ['FT9', 'CP1', 'PO9', 'Pz', 'CP2', 'P8', 'FT10', 'FP2']
channels_9  = ['Cz', 'C3', 'FC5', 'P3', 'P7','O2', 'T8','FC2']
channels_10 = ['Fp1','P4', 'P8', 'CP5', 'FC5', 'Fz', 'FC6', 'P7']
channels_11 = ['T7', 'O1', 'CP1', 'F4', 'FC6', 'CP2', 'P8', 'PO10']
channels_12 = ['FT9', 'Fp2', 'C3', 'F4', 'Pz', 'CP6', 'P7', 'F8']
channels_13 = ['PO9', 'Oz', 'T8', 'Fz', 'FT10', 'CP1', 'F4', 'CP6']
channels_14 = ['CP6', 'O2', 'P3', 'Cz', 'FC6', 'T7', 'FC5', 'Fp1']
#channels_15 = ['F3', 'C3', 'Cz', 'P3', 'CP2', 'CP6', 'F8', 'FC6']

init_pop = np.array([channels_0, channels_1, channels_2, channels_3, channels_4, channels_5, 
                    channels_6, channels_7, channels_8, channels_9, channels_10, channels_11, 
                    channels_12, channels_13, channels_14])

print(init_pop)
n_sub_ch = 8

[['F3' 'C3' 'P3' 'O1' 'O2' 'P4' 'C4' 'F4']
 ['Fp1' 'C3' 'P7' 'O1' 'O2' 'P8' 'C4' 'Fp2']
 ['Cz' 'Fz' 'P3' 'Pz' 'O1' 'Oz' 'O2' 'P4']
 ['Cz' 'Fz' 'T7' 'P3' 'O1' 'O2' 'P4' 'T8']
 ['F7' 'T7' 'P7' 'O1' 'O2' 'P8' 'T8' 'F8']
 ['FC1' 'FC5' 'FT9' 'CP5' 'CP1' 'PO9' 'T8' 'C4']
 ['F3' 'P7' 'PO10' 'CP2' 'CP6' 'FT10' 'FC6' 'FC2']
 ['Fz' 'F7' 'FC1' 'T7' 'Pz' 'P8' 'FC6' 'Fp2']
 ['FT9' 'CP1' 'PO9' 'Pz' 'CP2' 'P8' 'FT10' 'FP2']
 ['Cz' 'C3' 'FC5' 'P3' 'P7' 'O2' 'T8' 'FC2']
 ['Fp1' 'P4' 'P8' 'CP5' 'FC5' 'Fz' 'FC6' 'P7']
 ['T7' 'O1' 'CP1' 'F4' 'FC6' 'CP2' 'P8' 'PO10']
 ['FT9' 'Fp2' 'C3' 'F4' 'Pz' 'CP6' 'P7' 'F8']
 ['PO9' 'Oz' 'T8' 'Fz' 'FT10' 'CP1' 'F4' 'CP6']
 ['CP6' 'O2' 'P3' 'Cz' 'FC6' 'T7' 'FC5' 'Fp1']]


In [6]:
'Fp1' 'P4' 'P8' 'CP5' 'FC6' 'T7' 'FC5' 'Fp1'
def get_subset(data, all_channels, subset_channels):
    subset_data = np.empty((120, 8, 3200))

    j = 0
    for i in range(len(all_channels)):
        if j < (n_sub_ch + 1) and all_channels[i] in subset_channels:
            subset_data[:,j,:] = data[:,i,:]
            j+=1
    return subset_data

In [7]:
def check_nan(array):
    x = np.isnan(array)
    if True in x:
        print('NAN in array')
        return 0
    print('No NAN found')

In [8]:
def create_labels(dataset):
    labels = load_labels()
    label = pd.concat([labels['t1_math'], labels['t2_math'],
                    labels['t3_math']]).to_numpy()
    label = label.repeat(dataset.shape[1])
    return label

In [9]:
def convert_pop_to_fitness(current_pop,label):
    data = np.empty((len(current_pop), 3000, 24))
    new_pop_fitness = np.empty((len(current_pop),3))

    for i in range(len(current_pop)):
        subset_data = get_subset(dataset_ica_2_, channels, current_pop[i])
        dataset = convert_to_epochs(subset_data, n_sub_ch, v.SFREQ)
        features = time_series_features(dataset, n_sub_ch)
        data[i] = features
        check_nan(data[i])

        new_pop_fitness[i] = SVM(data[i], label)

    return new_pop_fitness

In [10]:
def convert_parents_to_fitness(current_pop,label, num_parents_mating):
    data = np.empty((num_parents_mating, 3000, 24))
    new_pop_fitness = np.empty((num_parents_mating,3))

    for i in range(num_parents_mating):
        subset_data = get_subset(dataset_ica_2_, channels, current_pop[i])
        dataset = convert_to_epochs(subset_data, n_sub_ch, v.SFREQ)
        features = time_series_features(dataset, n_sub_ch)
        data[i] = features
        check_nan(data[i])

        new_pop_fitness[i] = SVM(data[i], label)

    return new_pop_fitness

In [11]:
num_generations = 15
equation_inputs = [4, 1, 1]  # weight for accuracy, sensitivity and specificity 
pop_size = init_pop.shape[0]
print(pop_size)
curr_pop = init_pop
num_parents_mating = 5


# Creating labels
subset_data = get_subset(dataset_ica_2_, channels, init_pop[0])
dataset = convert_to_epochs(subset_data, n_sub_ch, v.SFREQ)
label = create_labels(dataset)

for generation in range(num_generations):
     # Measuring the fitness of each chromosome in the population.
     curr_pop_fitness = convert_pop_to_fitness(curr_pop,label)
     fitness = ga.cal_pop_fitness(equation_inputs, curr_pop_fitness)
     print(fitness)
     
    # Selecting the best parents in the population for mating.
     parents = ga.select_mating_pool(curr_pop, fitness, num_parents_mating)
 
     # Generating next generation using crossover.
     offspring_crossover = ga.crossover(parents, offspring_size=(pop_size-num_parents_mating, n_sub_ch))

     # Creating the new population based on the parents and offspring.
     curr_pop[0:num_parents_mating, :] = parents
     curr_pop[num_parents_mating:, :] = offspring_crossover

15
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
[3.59649817 3.59182855 3.70072328 3.59267375 3.64012284 3.45049569
 3.77926053 3.53388124 3.56300576 3.58485317 3.58945648 3.50293705
 3.72370481 3.586121   3.74603176]
[['' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '']
 ['' '' '' '' '' '' '' '']]
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 

In [12]:
print(parents)
print(offspring_crossover)

[['O2' 'T7' 'FC6' 'F8' 'FC2' 'CP6' 'CP2' 'P7']
 ['F8' 'CP2' 'FC2' 'FC6' 'O2' 'CP6' 'P7' 'T7']
 ['T7' 'FC2' 'CP2' 'P7' 'O2' 'CP6' 'F8' 'FC6']
 ['O2' 'CP2' 'CP6' 'T7' 'FC2' 'FC6' 'F8' 'P7']
 ['P7' 'FC6' 'F8' 'O2' 'T7' 'FC2' 'CP2' 'CP6']]
[['F8' 'FC2' 'P7' 'CP6' 'CP2' 'FC6' 'O2' 'T7']
 ['F8' 'O2' 'P7' 'CP6' 'T7' 'FC6' 'FC2' 'CP2']
 ['O2' 'CP6' 'CP2' 'T7' 'FC6' 'F8' 'FC2' 'P7']
 ['O2' 'FC6' 'F8' 'T7' 'FC2' 'CP2' 'CP6' 'P7']
 ['FC6' 'T7' 'P7' 'CP2' 'CP6' 'FC2' 'F8' 'O2']
 ['P7' 'F8' 'FC6' 'O2' 'CP6' 'T7' 'FC2' 'CP2']
 ['P7' 'FC2' 'CP6' 'CP2' 'O2' 'T7' 'FC6' 'F8']
 ['T7' 'F8' 'CP2' 'CP6' 'FC2' 'P7' 'O2' 'FC6']
 ['P7' 'CP6' 'FC6' 'O2' 'T7' 'CP2' 'F8' 'FC2']
 ['O2' 'P7' 'T7' 'FC2' 'FC6' 'CP6' 'CP2' 'F8']]


In [13]:
fitness = convert_parents_to_fitness(parents, label, num_parents_mating)

print(fitness)

(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
(120, 25, 8, 128)
No NAN found
[[0.61860465 0.68030691 0.784689  ]
 [0.61860465 0.68030691 0.784689  ]
 [0.61860465 0.68030691 0.784689  ]
 [0.61860465 0.68030691 0.784689  ]
 [0.61860465 0.68030691 0.784689  ]]
